In [40]:
import mne
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
mne.set_log_level(40)

In [41]:
def update_channels(raw):
    raw.drop_channels(['064'])
    update_names = ['EEG{:03d}'.format(i+1) for i in range(63)]
    map = dict(zip(raw.info['ch_names'], update_names))
    mne.rename_channels(raw.info, mapping=map)
    update_types = ['eeg'] * 60 + ['eog'] * 2 + ['ecg']
    map = dict(zip(raw.info['ch_names'], update_types))
    raw.set_channel_types(map)
    raw.set_montage('mgh60')
    return raw

In [42]:
def ica_filter(raw):
    raw = raw.filter(l_freq=.5, h_freq=35)
    ica = mne.preprocessing.ICA(
        n_components=20,
        max_iter='auto',
        random_state=10
    )
    ica.fit(raw)
    ecg_indices, ecg_scores = ica.find_bads_ecg(raw, method='correlation', threshold='auto')
    eog_indices, ecg_scores = ica.find_bads_eog(raw, threshold='auto')
    ica.exclude = ecg_indices + eog_indices
    epochs = ica.apply(raw, exclude=ica.exclude)
    epochs = mne.make_fixed_length_epochs(epochs, duration=30, preload=True)
    return epochs

In [43]:
for f in tqdm(os.listdir('data/ds003944')):
    if f.endswith('.set'):
        raw = mne.io.read_raw_eeglab('data/ds003944/'+f, preload=True)
        raw = update_channels(raw)
        # epochs = ica_filter(raw.copy())
        # epochs.save('data/processed/epochs/'+f.replace('.set', '.fif'))
        break

  1%|          | 1/144 [00:00<00:58,  2.43it/s]


In [44]:
raw = raw.filter(l_freq=.5, h_freq=35)

In [45]:
eog_evoked = mne.preprocessing.create_eog_epochs(raw).average()
eog_evoked.apply_baseline(baseline=(None, -0.2))
# eog_evoked.plot_joint()

<Evoked | '998' (average, N=97), -0.5 – 0.5 sec, baseline -0.5 – -0.2 sec, 60 ch, ~555 kB>

In [46]:
ecg_evoked = mne.preprocessing.create_ecg_epochs(raw).average()
ecg_evoked.apply_baseline(baseline=(None, -0.2))
# ecg_evoked.plot_joint()

<Evoked | '999' (average, N=220), -0.5 – 0.5 sec, baseline -0.5 – -0.2 sec, 60 ch, ~555 kB>

In [47]:
ica = mne.preprocessing.ICA(n_components=20, max_iter='auto', random_state=10)

In [48]:
ica.fit(raw.copy())

Method,fastica
Fit,54 iterations on raw data (306000 samples)
ICA components,20
Available PCA components,60
Channel types,eeg
ICA components marked for exclusion,—


In [49]:
%matplotlib inline
# ica.plot_components()

In [50]:
ecg_indices, ecg_scores = ica.find_bads_ecg(raw, method='correlation', threshold='auto')
eog_indices, ecg_scores = ica.find_bads_eog(raw, threshold='auto')

In [51]:
ica.exclude = ecg_indices + eog_indices
epochs = ica.apply(raw.copy(), exclude=ica.exclude)